In [1]:
# Chamar o arquivo df_cm_ddd:

import pandas as pd
resultados = pd.read_csv('./data/df_cm_ddd.csv', sep=';', encoding='utf-8')

In [2]:
# Transformações na base:

resultados['media_ano'] = resultados['media_ano'].map(lambda x: float(x.replace(',', '.')))               # Trocar a vírgula por ponto para transformar o número em float
resultados['Grupo Econômico'] = resultados['Grupo Econômico'].map(lambda x: str(x.replace('TELEFÔNICA', 'TELEFONICA')))  # Deixar todos TELEFONICA sem acento
resultados = resultados[resultados['Grupo Econômico'] != '0']    # Retirar o grupo econômico = 0

In [3]:
# Todas as outras operadoras serão "OUTROS"

# Crie um dicionário com as substituições desejadas
substituicoes = {
    'DATORA': 'OUTROS',
    'NEXTEL': 'OUTROS',
    'PORTO SEGURO': 'OUTROS',
    'UNICEL': 'OUTROS',
    'ALGAR (CTBC TELECOM)': 'OUTROS',
    'PREFEITURA DE LONDRINA/COPEL': 'OUTROS',
    'LIGGA TELECOM': 'OUTROS',
    'AMERICA NET': 'OUTROS',
    'SURF TELECOM': 'OUTROS',
    'EB FIBRA': 'OUTROS',
    'UNIFIQUE': 'OUTROS'
}

# Use a função replace() para aplicar as substituições ao DataFrame
resultados['Grupo Econômico'] = resultados['Grupo Econômico'].replace(substituicoes)

In [4]:
# resultados.info()

In [5]:
# resultados['Grupo Econômico'].unique()

In [6]:
# Escolher a operadora e dar o nome para o arquivo que será gerado no final
escolha_grupo = 'OI'
nome_arquivo_pkl = 'mapa_Oi.pkl'
resultados = resultados[resultados['Grupo Econômico'] == escolha_grupo]

# OI - 'mapa_Oi.pkl'
# TELEFONICA - mapa_Telefonica_Vivo.pkl
# TELECOM AMERICAS - 'mapa_TelecomAmericas_Claro.pkl'
# TELECOM ITALIA - 'mapa_TelecomItalia_Tim.pkl'
# OUTROS - 'mapa_Outros.pkl'

In [7]:
#criando dataframe com informações de georreferenciamento de municípios
georreferenciamento_df = pd.read_csv('https://raw.githubusercontent.com/kelvins/Municipios-Brasileiros/main/csv/municipios.csv')

#os dados de georreferenciamento tem 7 dígitos (vamos remover o dígito verificador e atualizar o dataframe)
georreferenciamento_df['codigo_ibge'] = georreferenciamento_df['codigo_ibge']//10

In [8]:
# georreferenciamento_df.head()

In [9]:
#cruzamento do dataframe resultados com as informações de georreferenciamento
resultados_df = pd.merge(resultados[['Código Nacional', 'ano', 'media_ano']],
                         georreferenciamento_df[['ddd', 'nome', 'codigo_ibge', 'codigo_uf', 'latitude', 'longitude']],
                         left_on='Código Nacional',
                         right_on='ddd',
                         how='inner')

In [10]:
# resultados_df.head()

In [11]:
#puxar a malha geográfica do brasil a nível de município
import requests
geojson = requests.get(f'http://servicodados.ibge.gov.br/api/v3/malhas/paises/BR?formato=application/vnd.geo+json&qualidade=minima&intrarregiao=municipio').json()

#a malha geográfica do ibge tem 7 dígitos (vamos remover o dígito verificador e atualizar a malha)
from geojson_rewind import rewind
for feature in geojson['features']:
    feature['properties']['codarea'] = feature['properties']['codarea'][:-1]
geojson = rewind(geojson, rfc7946=False)

In [12]:
# Função para encontrar o valor mínimo de todos os máximos da média_ano. Isto é usado para fixar a escala do mapa

max_min = resultados_df.groupby('ano').agg({'media_ano':'max'}).min().values[0]
resultados_df['media_ano'] = resultados_df['media_ano'].map(lambda x:max_min if x > max_min else x)

In [13]:
# Função para colocar diferentes cores nos mapas conforme seleção do grupo econômico

def cor_grupos(escolha_grupo):
    if escolha_grupo == 'TELECOM AMERICAS':
        result = 'amp'
    elif escolha_grupo == 'TELECOM ITALIA':
        result = 'ice_r'
    elif escolha_grupo == 'TELEFONICA':
        result = 'Purp'
    elif escolha_grupo == 'OI':
        result = 'ylgn'
    else:
        result = 'gray_r'      # gray_r para fazer a escala de cinzas invertido
    return result


In [14]:
#construir o mapa choroplético com timeline no campo de ano

import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'iframe'

fig = px.choropleth(resultados_df,
                    geojson=geojson,
                    # scope='south america',
                    color='media_ano',
                    color_continuous_scale=cor_grupos(escolha_grupo),
                    locations='codigo_ibge',
                    featureidkey='properties.codarea',
                    hover_name='nome',
                    animation_frame='ano')

fig.update_layout(height=800, width=1000, autosize=False,)
fig.update_geos(fitbounds = "locations", visible = False)
fig.update_traces(marker_line_width=0.05)

fig.show();

In [15]:
# # Construir gráficos em formato pickle para levar para o streamlit

# import pickle
# # nome_arquivo_pkl foi definido lá em cima quando escolhemos a operadora
# with open (nome_arquivo_pkl, 'wb') as pickle_file:
#     pickle.dump(fig, pickle_file)

In [16]:
# # Para zipar os arquivos pickel

# import gzip

# # Use gzip para criar um arquivo compactado (zipado) com extensão .gz
# with gzip.open(nome_arquivo_pkl + '.gz', 'wb') as f:
#     pickle.dump(fig, f)